In [1]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code')

In [3]:
!pip install setuptools~=67.6.0
!pip install spacy~=3.5.0
!pip install numpy~=1.21.5
!pip install gensim~=4.1.2
!pip install networkx~=2.8.4
!pip install tomotopy
!pip install corextopic
!pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.3.0 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found exi

In [15]:
import network_creation
from gensim.models.phrases import Phraser, ENGLISH_CONNECTOR_WORDS
import preprocessing
import community_utils
import tomotopy as tp
import networkx as nx
import igraph as ig
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
from diversity_metrics import *

In [81]:
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/text_datasets/europarl_es_train.txt", "r") as f:
    bbc_train = f.read().split("\n")
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/text_datasets/europarl_es_test.txt", "r") as f:
    bbc_test = f.read().split("\n")

In [82]:
import pickle
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/ep_es_master_object.obj", "rb") as f:
      master_object = pickle.load(f)

In [83]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt

import corextopic.corextopic as ct
import corextopic.vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [84]:
# Transform 20 newsgroup data into a sparse matrix
vectorizer = CountVectorizer(stop_words=None, max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(bbc_train)
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

(19001, 20000)

In [87]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names_out()))

In [88]:
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words

(19001, 19500)

In [89]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=5, words=words, max_iter=200, verbose=False, seed=1)
topic_model.fit(doc_word, words=words);

In [94]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

0: de, en, la, que, el, los, del, las, con, comisión
1: señor, presidente, sr, comisario, señora, informe, presidenta, sra, nombre, me
2: para, se, fondos, medio, ambiente, ayudas, no, medidas, sino, empresas
3: años, últimos, kosovo, hace, pero, reino, meses, desde, contra, unido
4: estados, miembros, una, un, entre, social, más, servicios, empleo, política


In [95]:
topic_words = []

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
  words,_,_ = zip(*topic)
  words_list = [i for i in words]
  topic_words.append(words_list)

print(topic_words)

[['de', 'en', 'la', 'que', 'el', 'los', 'del', 'las', 'con', 'comisión'], ['señor', 'presidente', 'sr', 'comisario', 'señora', 'informe', 'presidenta', 'sra', 'nombre', 'me'], ['para', 'se', 'fondos', 'medio', 'ambiente', 'ayudas', 'no', 'medidas', 'sino', 'empresas'], ['años', 'últimos', 'kosovo', 'hace', 'pero', 'reino', 'meses', 'desde', 'contra', 'unido'], ['estados', 'miembros', 'una', 'un', 'entre', 'social', 'más', 'servicios', 'empleo', 'política']]


In [96]:
print("puw:", proportion_unique_words(topic_words, topk=10))
print("jd:", pairwise_jaccard_diversity(topic_words, topk=10))
print("irbo p=0.5:", irbo(topic_words, weight=0.5, topk=10))
print("irbo p=0.9:", irbo(topic_words, weight=0.9, topk=10))


for coherence in ["c_v", "c_npmi", "u_mass"]:
  for topn in [5]:
    cm = CoherenceModel(topics=topic_words,
                        texts=master_object["ep_test"],
                        dictionary=master_object["ep_dict"],
                        topn=topn,
                        coherence=coherence)
    score = cm.get_coherence()
    print(coherence, ":", score, "(topn=", topn, ")")

puw: 1.0
jd: 1.0
irbo p=0.5: 1.0
irbo p=0.9: 1.0
c_v : 0.3536428689604272 (topn= 5 )
c_npmi : -0.1610589444235469 (topn= 5 )
u_mass : -8.591793703300782 (topn= 5 )
